https://www.kaggle.com/competitions/spaceship-titanic/overview

In [1]:
import sys
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
sys.path.insert(0, '../')
from utils import factorize
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe


Preprocessing


In [2]:
train_data = pd.read_csv('Data/train.csv')
validation_data = pd.read_csv('Data/test.csv')

val_ids = validation_data['PassengerId']
train_ids = train_data['PassengerId']

entire_data = pd.concat([train_data, validation_data], axis=0, sort=False)


entire_data = factorize(entire_data, exclude = ['PassengerId', 'Transported'])

validation_data = entire_data[entire_data['PassengerId'].isin(val_ids)].drop('Transported',axis=1)
train_data = entire_data[entire_data['PassengerId'].isin(train_ids)]

train_data = train_data.fillna(train_data.mean())
validation_data = validation_data.fillna(validation_data.mean())


META_COLS = ['PassengerId', 'Name']

train_data = train_data.drop(META_COLS, axis=1)

train_y = train_data['Transported']
train_x = train_data.drop('Transported', axis=1)

In [3]:
X_train, X_test, y_train, y_test= train_test_split(train_x, train_y, test_size=0.1, random_state=42)


lr = LogisticRegression()

lr.fit(X_train, y_train)


lr.score(X_train,y_train), lr.score(X_test, y_test)

/Users/adithyapalle/anaconda3/envs/kaggle/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.7732327751501982, 0.7528735632183908)

In [4]:

def write_predictions(model,fp = 'submission.csv'):

    pred = model.predict(validation_data.drop(META_COLS ,axis=1))

    pred_df = pd.DataFrame()

    pred_df['PassengerId'] = validation_data['PassengerId']

    pred_df['Transported'] = pred

    pred_df.to_csv(fp,index=False)

In [5]:

write_predictions(lr, fp='submission_lr.csv')

XGboost

In [6]:
x_train, x_val, y_train0, y_val0 = train_test_split(X_train, y_train, test_size=0.1) 

print(len(y_train0), len(y_val0),len(y_train))
xgb = XGBClassifier()

xgb.fit(X_train, y_train)


train_pred = xgb.predict(X_train)
test_pred = xgb.predict(X_test)



accuracy_score(y_train, train_pred), accuracy_score(y_test, test_pred) 

7040 783 7823


(0.9130768247475393, 0.7758620689655172)

Bayesian Optimization to reduce overfitting

In [7]:

def init_xgb_from_params(space):
    
    for key, value in space.items():
        try:
            space[key] = int(space[key])
        except Exception:  
            continue

    clf=XGBClassifier(
                    **space)
    
    return clf

In [8]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': hp.quniform('n_estimators',50,500,50),
        'seed': 0
    }


def objective(space):
    
    clf  = init_xgb_from_params(space)
    evaluation = [( x_train, y_train0), ( x_val, y_val0)]
    
    clf.fit(x_train, y_train0,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK } # just optimziign accuracy for now


trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)




  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

/Users/adithyapalle/anaconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

/Users/adithyapalle/anaconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                 
0.7597701149425288                                     
  1%|          | 1/100 [00:00<00:34,  2.88trial/s, best loss: -0.7597701149425288]

/Users/adithyapalle/anaconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

/Users/adithyapalle/anaconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                            
0.7505747126436781                                                                
  2%|▏         | 2/100 [00:00<00:28,  3.45trial/s, best loss: -0.7597701149425288]

...

SCORE:                                                                            
0.7609195402298851                                                                
 98%|█████████▊| 98/100 [00:24<00:00,  3.58trial/s, best loss: -0.767816091954023]

/Users/adithyapalle/anaconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

/Users/adithyapalle/anaconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                            
0.7574712643678161                                                                
 99%|█████████▉| 99/100 [00:25<00:00,  3.66trial/s, best loss: -0.767816091954023]

/Users/adithyapalle/anaconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

/Users/adithyapalle/anaconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                            
0.767816091954023                                                                 
100%|██████████| 100/100 [00:25<00:00,  3.94trial/s, best loss: -0.767816091954023]


In [9]:
print(best_hyperparams)
xgb = init_xgb_from_params(best_hyperparams)

xgb.fit(X_train, y_train) # fit on ALL TRAIN DATA
test_pred = xgb.predict(X_test)
train_pred = xgb.predict(X_train)
accuracy_score(y_train, train_pred) ,accuracy_score(y_test, test_pred) 

{'colsample_bytree': 0.5753966092225853, 'gamma': 3.3021835582533443, 'max_depth': 6.0, 'min_child_weight': 2.0, 'n_estimators': 200.0, 'reg_alpha': 81.0, 'reg_lambda': 0.20466227685256955}


(0.7739997443436022, 0.7632183908045977)